Import and declare variables

In [24]:
import subprocess
import os

data_root = "../data"

full_reference_genome = f"{data_root}/hg19.fasta"
reference_genome = f"{data_root}/hg19_targeted.fasta"
reference_genome_dict = f"{data_root}/hg19_targeted.dict"
reference_genome_fai = f"{data_root}/hg19_targeted.fasta.fai"
full_input_bam = f"{data_root}/input_reads.bam"
input_bam = f"{data_root}/input_reads_targeted.bam"
input_sam = f"{data_root}/input_reads_targeted.sam"
output_vcf = f"{data_root}/output.vcf"
genes_location = f"{data_root}/location.bed"

if not os.path.isfile(full_reference_genome):
    raise FileNotFoundError("reference genome is missing, add it please")
if not os.path.isfile(full_input_bam):
    raise FileNotFoundError("patient input bam file is missing, add it please")

Define gene locations

In [19]:
import pybedtools
if not os.path.isfile(genes_location):
    brca1_coordinates = pybedtools.create_interval_from_list(["chr17", "43044295", "43125482"])
    brca2_coordinates = pybedtools.create_interval_from_list(["chr13", "32315474", "32400266"])
    regions = [brca1_coordinates, brca2_coordinates]
    pybedtools.BedTool(regions).saveas(genes_location)

Extract target zone from reference genome

In [20]:
import subprocess
subprocess.run([f"bedtools getfasta -fi {full_reference_genome} -bed {genes_location} -fo {reference_genome}"], shell=True)

CompletedProcess(args=['bedtools getfasta -fi ../data/hg19.fasta -bed ../data/location.bed -fo ../data/hg19_targeted.fasta'], returncode=0)

Extract target zones from patient genome

In [21]:
import subprocess
subprocess.run([f"bedtools intersect -a {full_input_bam} -b {genes_location} > {input_bam}"], shell=True)

CompletedProcess(args=['bedtools intersect -a ../data/input_reads.bam -b ../data/location.bed > ../data/input_reads_targeted.bam'], returncode=0)

Generate sam from bam for visual check

In [27]:
import pysam
bam_file = pysam.AlignmentFile(input_bam, 'rb')
sam_file = pysam.AlignmentFile(input_sam, 'wh', header=bam_file.header)
for alignment in bam_file:
    sam_file.write(alignment)
bam_file.close()
sam_file.close()

In [28]:
if not os.path.isfile(reference_genome_fai):
    subprocess.run([f"samtools faidx {reference_genome}"], shell=True)

Indexing the input bam file

In [29]:
subprocess.run([f"samtools index {input_bam}"], shell=True)

CompletedProcess(args=['samtools index ../data/input_reads_targeted.bam'], returncode=0)

Create dict file required for VCF file generation

In [30]:
if not os.path.isfile(reference_genome_dict):
    subprocess.call([f"gatk CreateSequenceDictionary -R {reference_genome} -O {reference_genome_dict}"], shell=True)

Using GATK jar /Users/dascal/opt/anaconda3/envs/mamarCancer/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /Users/dascal/opt/anaconda3/envs/mamarCancer/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar CreateSequenceDictionary -R ../data/hg19_targeted.fasta -O ../data/hg19_targeted.dict
10:57:44.588 INFO  NativeLibraryLoader - Loading libgkl_compression.dylib from jar:file:/Users/dascal/opt/anaconda3/envs/mamarCancer/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.dylib


Tool returned:
0


[Sat Jul 01 10:57:44 EEST 2023] CreateSequenceDictionary --OUTPUT ../data/hg19_targeted.dict --REFERENCE ../data/hg19_targeted.fasta --TRUNCATE_NAMES_AT_WHITESPACE true --NUM_SEQUENCES 2147483647 --VERBOSITY INFO --QUIET false --VALIDATION_STRINGENCY STRICT --COMPRESSION_LEVEL 2 --MAX_RECORDS_IN_RAM 500000 --CREATE_INDEX false --CREATE_MD5_FILE false --help false --version false --showHidden false --USE_JDK_DEFLATER false --USE_JDK_INFLATER false
[Sat Jul 01 10:57:44 EEST 2023] Executing as dascal@Mihais-MacBook-Pro.local on Mac OS X 12.4 x86_64; OpenJDK 64-Bit Server VM 17.0.3+7-LTS; Deflater: Intel; Inflater: Intel; Provider GCS is available; Picard version: Version:4.4.0.0
[Sat Jul 01 10:57:45 EEST 2023] picard.sam.CreateSequenceDictionary done. Elapsed time: 0.01 minutes.
Runtime.totalMemory()=125829120


In [31]:
subprocess.call([f"gatk HaplotypeCaller -R {reference_genome} -I {input_bam} -O {output_vcf}"], shell=True)

Using GATK jar /Users/dascal/opt/anaconda3/envs/mamarCancer/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /Users/dascal/opt/anaconda3/envs/mamarCancer/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar HaplotypeCaller -R ../data/hg19_targeted.fasta -I ../data/input_reads_targeted.bam -O ../data/output.vcf
10:57:59.322 INFO  NativeLibraryLoader - Loading libgkl_compression.dylib from jar:file:/Users/dascal/opt/anaconda3/envs/mamarCancer/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.dylib
10:57:59.500 INFO  HaplotypeCaller - ------------------------------------------------------------
10:57:59.506 INFO  HaplotypeCaller - The Genome Analysis Toolkit (GATK) v4.4.0.0
10:57:59.506 INFO  HaplotypeCaller - For support and documentation go to https://software.broa

2

In [ ]:
import pysam
import matplotlib.pyplot as plt

vcf_reader = pysam.VariantFile(open(output_vcf, "r"))

# Păstrați informațiile relevante pentru grafic (de exemplu, AF - frecvența alelei alternative)
allele_frequencies = []
for record in vcf_reader:
    allele_frequencies.append(record.info.get("AF")[0])

# Plasați graficul
plt.plot(allele_frequencies)
plt.xlabel("Variant Index")
plt.ylabel("Allele Frequency")
plt.title("VCF Variant Allele Frequency")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Lista pentru a stoca calitățile
qualities = []

# Citirea fișierului VCF și extragerea calităților
with open(output_vcf, 'r') as file:
    for line in file:
        if not line.startswith('#'):
            data = line.strip().split('\t')
            quality = float(data[5])
            qualities.append(quality)

# Crearea histogramă a calităților
plt.hist(qualities, bins=20, edgecolor='black')

# Etichetele axelor
plt.xlabel('Quality')
plt.ylabel('Count')

# Titlul diagramei
plt.title('Quality Histogram')

# Afișarea diagramei
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Dicționar pentru a stoca frecvențele alelelor
allele_frequencies = {}

# Citirea fișierului VCF și extragerea informațiilor despre frecvențele alelelor
with open(output_vcf, 'r') as file:
    for line in file:
        if not line.startswith('#'):
            data = line.strip().split('\t')
            info = data[7].split(';')
            for item in info:
                if item.startswith('AF='):
                    freq = item.split('=')[1]
                    if len(item.split(',')) > 1:
                        continue
                    print(freq)
                    allele_frequency = float(freq)
                    allele_frequencies['ALT'] = allele_frequency
                    allele_frequencies['REF'] = 1 - allele_frequency

# Crearea diagramă de tip "pie"
labels = ['Reference (REF)', 'Alternate (ALT)']
sizes = [allele_frequencies['REF'], allele_frequencies['ALT']]
colors = ['#1f77b4', '#ff7f0e']
explode = (0.1, 0)  # Pentru a separa puțin secțiunea "Reference"

plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)

# Aspectul circular
plt.axis('equal')

# Titlul diagramei
plt.title('Allele Frequencies')

# Afișarea diagramei
plt.show()
